In [1]:
import numpy as np
import pandas as pd
from osgeo import gdal
from gdalconst import *
#import osr
import matplotlib.pyplot as plt
%matplotlib qt




/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#Read a raster
Test_data = ("/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/")
Test = gdal.Open(Test_data + "/Mandakini.bil")#Get raster data
M_DEM = Test.ReadAsArray()

#Read stream data
name = "Mandakini_fullProfileMC_forced_0.45_3_1258909000_10_80_281_for_Arc.csv" #Set file name
path = "/Users/bmelosh/VagrantBoxes/LSDTopoTools/Topographic_projects/Test_data/" #Set path to data
#for csv files
data = np.genfromtxt((path+name), delimiter=',', skip_header=1, names=['id', 'x', 'y', 'chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])
#for tree files
#data = np.genfromtxt((path+name), delimiter=' ', skip_header=1, names=['chan_number', 'reciever_chan','node_on_reciever_chan', 'node', 'row','column', 'flow_distance', 'chi', 'elevation', 'drainage_area', 'n_data_points', 'm_mean','m_st_dev', 'm_std_err', 'b_mean', 'b_st_dev', 'b_std_err', 'DW_mean', 'DW_st_dev', 'DW_std_err', 'fitted_elev_mean', 'fitted_elev_stdev', 'fitted_elev_std_err'])


In [3]:



def plot_coord(Test):

    (upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
    dx = x_size
    dy = y_size
    nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

    xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
    yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

    #Create arrays of the x and y coordinates of each pixel (the axes)
    xcoordinates = [x*dx + xllcenter for x in range(nx)]
    ycoordinates = [y*dy + yllcenter for y in range(ny)]

    #Create 2 2d grids describing x and y coordinates
    X,Y = np.meshgrid(xcoordinates, ycoordinates) 


    fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))

    return X, Y


In [52]:
(upper_left_x, x_size, x_rotation, upper_left_y, y_rotation, y_size) = Test.GetGeoTransform()
dx = x_size
dy = y_size
nx, ny = Test.RasterXSize, Test.RasterYSize  # Size of the original raster

xllcenter = upper_left_x + dx/2  # x coordinate center of lower left pxl
yllcenter = upper_left_y - dx/2 # y coordinate center of lower left pxl   - (ny-1)*dx 

#Create arrays of the x and y coordinates of each pixel (the axes)
xcoordinates = [x*dx + xllcenter for x in range(nx)]
ycoordinates = [y*dy + yllcenter for y in range(ny)]

#Create 2 2d grids describing x and y coordinates
X,Y = np.meshgrid(xcoordinates, ycoordinates) 


fig = plt.contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
    

In [10]:
[X, Y] = plot_coord(Test)

In [5]:
cm = plt.cm.gist_earth


fig, ax = plt.subplots(3, figsize=(10,10), sharex=False) #define 2 subfigures

ax[0].contourf(X, Y, M_DEM, levels=np.linspace(np.amin(M_DEM[M_DEM > 0]),np.amax(M_DEM), 50))
ax[0].plot(data['x'], data['y'], 'ro',  markersize=2)
#ax[0].imshow(fA, interpolation='nearest', vmin=0, cmap=plt.cm.gray) 
ax[0].set_aspect('equal')
ax[0].set_title('Terrain map with stream channel')
ax[0].set_ylabel('Northing')
ax[0].set_xlabel('Easting')


ax[1].scatter(data['flow_distance'], data['elevation'], c=data['elevation'], cmap=cm)
ax[1].set_title('Stream Map')
ax[1].set_ylabel('Elevation (m)')
ax[1].set_xlabel('Distance (m)')


ax[2].scatter(data['chi'], data['elevation'], c=data['elevation'], cmap=plt.get_cmap("jet"))
ax[2].text(3, 2500, 'm/n = 0.45')
ax[2].set_ylabel('Elevation (m)')
ax[2].set_xlabel('Chi (X)')


plt.show()

In [18]:
img = (X, Y)


In [6]:
#Import bokeh 
from bokeh.plotting import *
from bokeh.models import ColumnDataSource, Label
from bokeh.layouts import layout

#package in this working directory
from utils.colormap import RGBAColorMapper
import netCDF4
from bokeh.palettes import RdBu11
#import color palettes for plotting raster
from bokeh.palettes import *




In [118]:

#My bokeh figure
mn = " (m/n = 0.45)"

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
stream_source = ColumnDataSource(data=dict(x0=data['flow_distance'], x1=data['chi'], y=data['elevation']))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title='Stream Profile')
left.circle('x0', 'y', source=stream_source)
left.xaxis.axis_label = 'Flow Distance (m)'
left.yaxis.axis_label = 'Elevation (m)'

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title='Chi Profile'+ mn)
right.circle('x1', 'y', source=stream_source)#, legend = "m/n = 0.45")
right.xaxis.axis_label = 'Chi (X)'
right.yaxis.axis_label = 'Elevation (m)'


#Set ranges for DEM image
dw = max(data['x'])- min(data['x'])
dh = max(data['y'])- min(data['y'])
xrmax = max(data['x'])
xrmin = min(data['x'])
yrmax = max(data['y'])
yrmin = min(data['y'])

topo = figure(plot_height=700, plot_width=700, x_axis_type=None, y_axis_type=None, x_range=[xrmin,xrmax], y_range=[yrmin,yrmax])

topo.image(image=[dem], x=[min(data['x'])], y=[min(data['y'])],dw=[dw], dh=[dh], palette=pal, name='image')
topo.circle(data['x'], data['y'], color='red', size=3)


# put the subplots in a gridplot
p = layout([[left, right], [topo]])



show(p)


SyntaxError: invalid syntax (<ipython-input-118-f225e0bc54ad>, line 62)

In [125]:
import geopandas as gpd

grid = gpd.read_file(Test)

AttributeError: type object 'object' has no attribute '__getattr__'

In [33]:
#attempting to plot raster

import numpy as np

from bokeh.plotting import figure, show, output_file


xx, yy = np.meshgrid(data['x'], data['y'])
d = data['elevation']


p = figure(x_range=(0,100), y_range=(0,100))#(min(data['x']),max(data['x'])), y_range=(min(data['y']),max(data['y'])))
# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11")

output_file("image.html", title="image.py example")

show(p)  # open a b

In [34]:

from bokeh.plotting import figure, output_notebook, show
data = netCDF4.Dataset('Land_and_Ocean_LatLong1.nc')
t = data.variables['temperature']

colormap = RGBAColorMapper(-6, 6, RdBu11)
# Set output option
output_notebook()


# Write a function ``get_slice(temperature_variable, year, month)`` that given a temperature variable, year and month
# returns an array of rgba colors. (Hint: Use RGBAColorMapper.color()). 
def get_slice(t, year, month):
    i = (year - 1850)*12 + month - 1
    return colormap.color(t[i, :, :])

# Get image data for a given year and month using the ``get_slice`` function
image = get_slice(t, 1950, 1)




# Create plot
# Tip: Make sure to set x_range and y_range appropriately
p =  figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])

p.image_rgba(
    image=[image],
    x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)

q = figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])
q.image_rgba(image=[dem], 
            x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)


# Show plot
show(q)

Loading BokehJS ...

/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in less
/Users/bmelosh/anaconda2/envs/geospatial/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in greater


In [44]:
import numpy as np
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_file

# Sample Data
img = np.array([[(x/10.*255,x/10.*255,100,255) for x in range(10)] for y in range(10)])

# Convert to RGBA array that can be plotted
d = np.empty((10, 10), dtype=np.uint32)
view = d.view(dtype=np.uint8).reshape((10, 10, 4))
view[:,:,:] = img

# Set output file
output_file("image.html", title="image.py example")

# Setup the figure
rng = Range1d(0,10,bounds='auto')
p = figure(x_range=rng, y_range=rng, plot_width=500, plot_height=500,active_scroll='wheel_zoom')

# Slice the images
imgs = [d[:,n:n+1] for n in range(10)]
dhs = [10 for n in range(10)]
ys = [0 for n in range(10)]
dws = [0.5 if n%2 == 0 else 1.5 for n in range(10)]
xs = [sum(dws[:n]) for n in range(10)]

# Plot the image
p.image_rgba(image=imgs, x=xs, y=ys, dw=dws, dh=dhs)
show(p)

In [19]:
#img = np.array((len(X),len(Y)), dtype=np.unit32)
img =np.array((X,Y), dtype=np.uint32)

p = figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])

p.image_rgba(
    image=[img],
    x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)
show(p)

In [12]:

# Initialize the colormap with a bokeh palette and the low and high values for the scale.
#from bokeh.palettes import RdBu11
#colormap = RGBAColorMapper(-6, 6, RdBu11)

p =  figure(width=900, height=500, x_axis_type=None, y_axis_type=None, x_range=[-180,180], y_range=[-90,90])

p.image_rgba(
    image=[M_DEM],
    x=[-180], y=[-90],
    dw=[360], dh=[180], name='image'
)

show(p)

In [62]:
dem = M_DEM[::-1]

In [109]:
from bokeh.palettes import *


dw = max(data['x'])- min(data['x'])
dh = max(data['y'])- min(data['y'])

xrmax = max(data['x'])
xrmin = min(data['x'])
yrmax = max(data['y'])
yrmin = min(data['y'])

p = figure(plot_height=900, plot_width=900, x_axis_type=None, y_axis_type=None, x_range=[xrmin,xrmax], y_range=[yrmin,yrmax])


p.image(image=[dem], x=[min(data['x'])], y=[min(data['y'])],dw=[dw], dh=[dh], palette=pal, name='image')

#p.image(image=[dem], x=[min(data['x'])], y=[min(data['y'])],dw=[dw], dh=[dh], palette=g, name='image', alpha=0.5)

        #[max(data['x'])], y=[min(data['y'])], dw=[dw], dh=[dh], palette="Spectral11")
show(p)

In [108]:
#Create Color palette
pal = viridis(256)
pal[0] = '#000000'
g = gray(256)


In [60]:
yrmax = max(data['y'])
print(yrmax)
yrmin = min(data['y'])
print(yrmin)

xrmax = max(data['x'])
print(xrmax)
xrmin = min(data['x'])
print(xrmin)
dw

3377078.852
3372194.102
328128.8534
316316.5722


11812.281200000027

In [75]:
#Create arrays of the x and y coordinates of each pixel (the axes)
xcoordinates = [x*dx + xllcenter for x in range(nx)]
ycoordinates = [y*dy + yllcenter for y in range(ny)]

#Create 2 2d grids describing x and y coordinates
X,Y = np.meshgrid(xcoordinates, ycoordinates) 
xcoordinates
print(X[0][1])
print(Y[0][1])

im[0][2]

302106.305415
3410028.75865


290471.65247274149

In [21]:
N = 500
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(x_range=(0, 10), y_range=(0, 10))

# must give a vector of image data for image parameter
p.image(image=[M_DEM], x=0, y=0, dw=10, dh=10, palette="Spectral11")

output_file("image.html", title="image.py example")

show(p)  # open a browser